In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import plot
import firedrake
from firedrake import (Constant, as_vector, inner, sym, perp,
                       grad, div, dx, ds)

# The Stokes problem

In this demo, I'll look at how to solve the Stokes problem for very viscous fluid flow.
We'll use the same domain as the last example, but we'll imagine that the two holes in the center are actually counter-rotating cylinders that drag the fluid along with it.

In [ ]:
mesh = firedrake.Mesh('domain.msh')

I'll plot the mesh again because the numeric IDs of the boundary components will be important this time around.
We need to remember that IDs 5 and 6 are for the circular holes in the domain.
The holes are centered at $(-1/2, 1/4)$ and $(1/2, -1/4)$ with radii of $1/4$ and $3/8$, respectively.

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect('equal')
plot.triplot(mesh, axes=axes)
fig.show()

There are two key differences here with the Laplace equation.
First, we need to pick a *vector* function space for the velocities as well as a function space for the pressures.
Second, we have to be careful about the [LBB condition](https://en.wikipedia.org/wiki/Ladyzhenskaya%E2%80%93Babu%C5%A1ka%E2%80%93Brezzi_condition).
We've used continuous piecewise quadratic velocities and discontinuous piecewise constant pressures, which are known to satisfy the LBB conditions, but other choices are possible.

In [ ]:
V = firedrake.VectorFunctionSpace(mesh, family='CG', degree=2)
Q = firedrake.FunctionSpace(mesh, family='DG', degree=0)
Z = V * Q

Next we have to set a viscosity.
We'll choose a large value so that the Reynolds number is quite low.

In [ ]:
μ = firedrake.Constant(10)

Since we're using a mixed function space, we can call `TrialFunctions` instead of `TrialFunction` to get a pair of trial functions representing each component.
Likewise for test functions.

In [ ]:
u, p = firedrake.TrialFunctions(Z)
v, q = firedrake.TestFunctions(Z)

Defining the weak form is a similar concept to what we did before.
The constitutive relations for most fluids are described in terms of the rate-of-strain tensor

$$\dot\varepsilon(u) = \frac{1}{2}\left(\nabla u + \nabla u^*\right).$$

To make the code look as much like the math as possible, we'll introduce an auxiliary function `ε` to calculate the strain rate tensor.

In [ ]:
def ε(u):
    return sym(grad(u))

a = (2 * μ * inner(ε(u), ε(v)) - inner(p, div(v)) - inner(q, div(u))) * dx
F = inner(Constant(as_vector((0, 0))), v) * dx

The velocity field will rotate with a uniform speed of 1 around each of the circular holes inside the domain.
You could imagine that this boundary condition arises because there are two cylindrical rods spinning at this speed immeresed in the fluid.
The function `perp(v)` is equivalent to

$$v^\bot = \hat z \times v.$$

Finally, we'll make the fluid velocity equal to 0 around the outer domain boundary.

In [ ]:
x = firedrake.SpatialCoordinate(mesh)

x1 = as_vector((-1/2, 1/4))
r1 = 1/4
g1 = -perp(x - x1) / r1

x2 = as_vector((1/2, -1/4))
r2 = 3/8
g2 = perp(x - x2) / r2

In [ ]:
bc1 = firedrake.DirichletBC(Z.sub(0), g1, 5)
bc2 = firedrake.DirichletBC(Z.sub(0), g2, 6)
bc3 = firedrake.DirichletBC(Z.sub(0), as_vector((0., 0.)), (1, 2, 3, 4))

Getting a good linear solver for the Stokes equations is *much* more complex than for the Laplace equation because the underlying linear system is no longer positive-definite.
The following solver parameters specify that we are using:

* a matrix-free method (`matfree`) where we don't explicitly form a sparse matrix but rather apply matrix-vector products
* the GMRES method as an outer-level solver and a Schur complement preconditioner
* a multigrid preconditioner from the package [hypre](https://github.com/hypre-space/hypre) for the velocities
* the inverse mass matrix to precondition the pressures
* an incomplete LU factorization to precondition the mass matrix inverse solve

We also have to tell the solver that the operator has a null space, namely the velocities that are spatially constant.

In [ ]:
parameters = {
    'mat_type': 'matfree',
    'ksp_type': 'gmres',
    'pc_type': 'fieldsplit',
    'pc_fieldsplit_type': 'schur',
    'pc_fieldsplit_schur_fact_type': 'diag',
    'fieldsplit_0_ksp_type': 'preonly',
    'fieldsplit_0_pc_type': 'python',
    'fieldsplit_0_pc_python_type': 'firedrake.AssembledPC',
    'fieldsplit_0_assembled_pc_type': 'hypre',
    'fieldsplit_1_ksp_type': 'preonly',
    'fieldsplit_1_pc_type': 'python',
    'fieldsplit_1_pc_python_type': 'firedrake.MassInvPC',
    'fieldsplit_1_Mp_ksp_type': 'preonly',
    'fieldsplit_1_Mp_pc_type': 'ilu'
}

from firedrake import MixedVectorSpaceBasis, VectorSpaceBasis
nullspace = MixedVectorSpaceBasis(
    Z, [Z.sub(0), VectorSpaceBasis(constant=True)])

In order to solve the system, we need a variable that we'll call `z` to put the solution in.
This variable lives in the mixed space and aggregates together the velocities and pressures.
We're also using several boundary conditions and not just one.

In [ ]:
z = firedrake.Function(Z)
firedrake.solve(a == F, z, bcs=[bc1, bc2, bc3],
                nullspace=nullspace,
                solver_parameters=parameters)

To get the velocity and pressure back out of `z` we call the method `split`.

In [ ]:
u, p = z.split()

We can then plot and analyze these functions separately.

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect('equal')
contours = plot.tricontourf(u, 40, cmap='viridis', axes=axes)
fig.colorbar(contours)
fig.show()

As a sanity check, we can make sure that the divergence of the velocity field is small in some sense.

In [ ]:
div_norm = firedrake.assemble(div(u)**2 * dx)
grad_norm = firedrake.assemble(inner(grad(u), grad(u)) * dx)
print(div_norm / grad_norm)

In [ ]:
# TODO: tripcolor
fig, axes = plt.subplots()
contours = plot.tricontourf(p, 40, cmap='viridis', axes=axes)
fig.colorbar(contours)
fig.show()

### Saving the results

Firedrake has a way to save the result to an HDF5 file so that you can resume using it later.
This is called *checkpointing* and it's especially useful for long-running simulations where there's a chance it could crash in the middle or where you want to separate the analysis from the simulation.
You have to pass a name when you store a field and remember what the name was when you want to load it again.

In [ ]:
chk = firedrake.DumbCheckpoint('velocity', mode=firedrake.FILE_CREATE)
chk.store(u, name='u')

We'll use the resulting velocity in the next demo.

### Things to try

* Change the finite element spaces
* Change the relative velocities of the cylinders
* Make the cylinders rotate in the same direction